In [ ]:
import numpy as np
import pandas as pd
import heapq
import folium
from folium.plugins import HeatMap
from scipy.spatial import cKDTree
from scipy.interpolate import splprep, splev
from sklearn.preprocessing import MinMaxScaler
import datetime

# ---------------------------
# Global Configuration
# ---------------------------
WORLD_BOUNDS = {
    'lat_min': -90,
    'lat_max': 90,
    'lon_min': -180,
    'lon_max': 180
}

# ---------------------------
# Data Generation Module
# ---------------------------

class GlobalDataGenerator:
    def __init__(self, seed=42):
        np.random.seed(seed)
        
    def generate_bird_migrations(self, num_clusters=30, points_per_cluster=100):
        """Generate global bird migration patterns"""
        cluster_centers = np.column_stack([
            np.random.uniform(WORLD_BOUNDS['lat_min'], WORLD_BOUNDS['lat_max'], num_clusters),
            np.random.uniform(WORLD_BOUNDS['lon_min'], WORLD_BOUNDS['lon_max'], num_clusters)
        ])
        
        return np.vstack([
            np.random.normal(loc=center, scale=[5, 10], size=(points_per_cluster, 2))
            for center in cluster_centers
        ])
    
    def generate_weather_systems(self, num_systems=50):
        """Generate global weather systems"""
        systems = []
        start_date = datetime.datetime(2025, 3, 1)
        for _ in range(num_systems):
            systems.append([
                np.random.uniform(WORLD_BOUNDS['lat_min'], WORLD_BOUNDS['lat_max']),
                np.random.uniform(WORLD_BOUNDS['lon_min'], WORLD_BOUNDS['lon_max']),
                np.random.uniform(1, 10),
                start_date + datetime.timedelta(days=np.random.randint(0, 30))
            ])
        return pd.DataFrame(systems, columns=['latitude', 'longitude', 'severity', 'date'])

# ---------------------------
# Risk Modeling Module
# ---------------------------

class GlobalRiskModel:
    def __init__(self, grid_resolution=1.0):
        self.grid_resolution = grid_resolution
        self.lat_range = (WORLD_BOUNDS['lat_min'], WORLD_BOUNDS['lat_max'])
        self.lon_range = (WORLD_BOUNDS['lon_min'], WORLD_BOUNDS['lon_max'])
        self._create_global_grid()
        
    def _create_global_grid(self):
        """Create worldwide grid"""
        self.lat_grid = np.arange(*self.lat_range, self.grid_resolution)
        self.lon_grid = np.arange(*self.lon_range, self.grid_resolution)
        self.grid_shape = (len(self.lat_grid), len(self.lon_grid))
        self.risk_grid = np.ones(self.grid_shape)
        
    def add_global_risk(self, bird_data, weather_data, date):
        """Calculate global risk factors"""
        # Bird strike risk
        bird_tree = cKDTree(bird_data)
        for i, lat in enumerate(self.lat_grid):
            for j, lon in enumerate(self.lon_grid):
                count = len(bird_tree.query_ball_point([lat, lon], 500/111))  # 500 km radius
                self.risk_grid[i,j] += count * 0.2  # Scale factor
        
        # Weather risk with temporal decay
        weather_data['days_diff'] = (pd.to_datetime(date) - weather_data['date']).dt.days.abs()
        active_weather = weather_data[weather_data['days_diff'] <= 10]
        
        for _, row in active_weather.iterrows():
            lat_idx = np.abs(self.lat_grid - row['latitude']).argmin()
            lon_idx = np.abs(self.lon_grid - row['longitude']).argmin()
            decay = np.exp(-row['days_diff']/10)
            self.risk_grid[lat_idx, lon_idx] += row['severity'] * decay
        
        self._normalize_risks()
        
    def _normalize_risks(self):
        """Normalize risk scores between 1-10"""
        self.risk_grid = MinMaxScaler(feature_range=(1, 10)).fit_transform(self.risk_grid)

# ---------------------------
# Pathfinding Module
# ---------------------------

class ThetaStarOptimizer:
    def __init__(self, risk_model):
        self.risk_model = risk_model
        self.neighbors = [(-1,0), (1,0), (0,-1), (0,1),
                         (-1,-1), (-1,1), (1,-1), (1,1)]
        
    def find_path(self, start_coord, end_coord):
        start = self._coord_to_index(start_coord)
        end = self._coord_to_index(end_coord)
        
        open_heap = []
        heapq.heappush(open_heap, (0, start))
        came_from = {}
        cost_so_far = {start: 0}
        
        while open_heap:
            current = heapq.heappop(open_heap)[1]
            
            if current == end:
                return self._reconstruct_path(came_from, current)
                
            for neighbor in self._get_neighbors(current):
                if self._line_of_sight(came_from.get(current, None), neighbor):
                    new_cost = cost_so_far[current] + self._calculate_cost(current, neighbor)
                else:
                    new_cost = cost_so_far[current] + self._calculate_cost(current, neighbor)
                
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    priority = new_cost + self._heuristic(end, neighbor)
                    heapq.heappush(open_heap, (priority, neighbor))
                    came_from[neighbor] = current
        return None
    
    def _heuristic(self, a, b):
        return np.hypot(a[0]-b[0], a[1]-b[1])
    
    def _coord_to_index(self, coord):
        lat_idx = np.abs(self.risk_model.lat_grid - coord[0]).argmin()
        lon_idx = np.abs(self.risk_model.lon_grid - coord[1]).argmin()
        return (lat_idx, lon_idx)
    
    def _get_neighbors(self, node):
        return [(node[0]+dx, node[1]+dy) for dx, dy in self.neighbors
                if 0 <= node[0]+dx < self.risk_model.grid_shape[0] and
                0 <= node[1]+dy < self.risk_model.grid_shape[1]]
    
    def _line_of_sight(self, parent, node):
        """Bresenham's line algorithm implementation"""
        if parent is None:
            return False
        # Simplified for example - implement collision checking here
        return True
    
    def _calculate_cost(self, a, b):
        return self.risk_model.risk_grid[b]
    
    def _reconstruct_path(self, came_from, current):
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        return path[::-1]

# ---------------------------
# Path Smoothing
# ---------------------------

def smooth_path(path_coords, smoothing_factor=0.5):
    """Apply spline interpolation to path coordinates"""
    if len(path_coords) < 4:
        return path_coords
        
    tck, u = splprep(np.array(path_coords).T, s=smoothing_factor)
    new_points = splev(np.linspace(0, 1, 100), tck)
    return list(zip(new_points[0], new_points[1]))

# ---------------------------
# Visualization Module
# ---------------------------

class EnhancedVisualizer:
    def __init__(self, risk_model):
        self.risk_model = risk_model
        
    def create_map(self, path_indices, start_coord, end_coord):
        m = folium.Map(location=start_coord, zoom_start=3)
        
        # Add risk heatmap
        heat_data = [[lat, lon, risk] 
                    for i, lat in enumerate(self.risk_model.lat_grid)
                    for j, lon in enumerate(self.risk_model.lon_grid)
                    for risk in [self.risk_model.risk_grid[i,j]]]
        
        HeatMap(
            heat_data,
            gradient={'0.0': 'blue', '0.5': 'cyan', '0.8': 'yellow', '1.0': 'red'},  # String keys
            radius=20,
            blur=30,
            max_zoom=7
        ).add_to(m)
        
        # Add flight path
        if path_indices:
            raw_coords = [(self.risk_model.lat_grid[i], self.risk_model.lon_grid[j]) 
                         for i, j in path_indices]
            smooth_coords = smooth_path(raw_coords)
            folium.PolyLine(
                smooth_coords, 
                color='lime', 
                weight=4,
                opacity=0.8
            ).add_to(m)
        
        # Add markers
        folium.Marker(start_coord, 
                     icon=folium.Icon(color='green', icon='plane', prefix='fa')).add_to(m)
        folium.Marker(end_coord, 
                     icon=folium.Icon(color='red', icon='plane', prefix='fa')).add_to(m)
        
        # Add legend
        legend_html = '''
            <div style="position: fixed; 
                        bottom: 50px; left: 50px; width: 150px; 
                        z-index: 1000; background: white; padding: 10px;">
                <strong>Risk Legend</strong><br>
                <i style="background: blue"></i> Low (1-3)<br>
                <i style="background: cyan"></i> Moderate (4-6)<br>
                <i style="background: yellow"></i> High (7-9)<br>
                <i style="background: red"></i> Extreme (10)
            </div>'''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        return m

# ---------------------------
# Main Execution
# ---------------------------

def main():
    # User Input - Set your coordinates here!
    START_COORD = (31.9632, 35.9937)  
    END_COORD = (25.2532, 55.3657)   
    
    # Generate data
    data_gen = GlobalDataGenerator()
    birds = data_gen.generate_bird_migrations()
    weather = data_gen.generate_weather_systems()
    
    # Build risk model
    risk_model = GlobalRiskModel(grid_resolution=1.0)
    risk_model.add_global_risk(birds, weather, date='2025-03-15')
    
    # Find path
    pathfinder = ThetaStarOptimizer(risk_model)
    path = pathfinder.find_path(START_COORD, END_COORD)
    
    # Visualize
    if path:
        visualizer = EnhancedVisualizer(risk_model)
        m = visualizer.create_map(path, START_COORD, END_COORD)
        m.save('enhanced_flight_path.html')
        print("Map saved successfully!")
    else:
        print("No safe path found")

if __name__ == "__main__":
    main()

Map saved successfully!
